In [2]:
import pandas as pd
import numpy as np
import os
os.environ['MKL_NUM_THREADS'] = '1'
from collections import Counter
import itertools
import json
import scipy.sparse as sparse
import pickle
import torch
import misc.util as util
from torch import nn

import importlib
from torch.utils.data import DataLoader
from misc.loader import AEDataset
from misc.util import *
from tqdm.auto import tqdm
from eval.rec_eval import *
import neuralsort.pl as pl
from models.loss import neuPrecLoss
from misc.loader import RecDataset
import models
from models.loss import *

In [34]:
import pickle
with open("data/parsed/ml-1m-l-1-100", 'rb') as f:
    tr, val = pickle.load(f)

In [37]:
tr

<6034x2811 sparse matrix of type '<class 'numpy.float64'>'
	with 566160 stored elements in Compressed Sparse Row format>

In [38]:
n_users, n_items = tr.shape
ae_dataset = AEDataset(tr)
total_anneal_steps = 50000
anneal_cap = 0.2


In [40]:
from lightfm import LightFM
model = LightFM(no_components=32,
                learning_rate=1e-2,
                item_alpha=0.01,
                user_alpha=0.01,
                max_sampled=100,
                loss='bpr')
model.fit_partial(tr, epochs=10,)

In [30]:
ub, uv = model.get_user_representations()
ib, iv = model.get_item_representations()

In [31]:
w = util.get_wrapper(uv, iv, ub, ib)

In [33]:
leave_k_eval(w, tr, val, leavek=1, K=5)

{'precision': 0.3824991713622804,
 'map': 0.23115677825654685,
 'ndcg': 0.26854139867777893}

In [6]:
vin_dense = torch.FloatTensor(np.asarray(vin.todense())).cuda()
tin_dense = torch.FloatTensor(np.asarray(tin.todense())).cuda()

NameError: name 'vin' is not defined

In [7]:
list(zip([1,2],[3,4]))

[(1, 3), (2, 4)]

In [16]:
import models.ae
importlib.reload(models.ae)
model = models.ae.MultiDAE([200] + [n_items], n_users=n_users, dropout=0.5)
x = model.rank_items(3, tr, np.asarray([1,2,3]))
print(x)

[(2, 0.0126362275), (1, 0.0011439759), (3, -0.02918803)]


In [17]:
loader = DataLoader(ae_dataset, batch_size=256, shuffle=True, num_workers=4)

In [21]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in (range(1, 5 + 1)):
    model = model.train()
    model.training = True
    tr_losses = []
    for uid, row in  (loader):
        loss = None 
        scores = model.forward(row, uid)
        if total_anneal_steps > 0:
            anneal_cap = min(0.1, 1. * 1 / total_anneal_steps)
        else:
            anneal_cap = 0.1
            
        loss  = models.loss.MSELoss(row, scores)
        model.zero_grad()
        (loss.mean()).backward()
        tr_losses.append(loss.detach().unsqueeze(-1)) 
        optimizer.step()
        print(loss)

tensor([117.4600, 286.4197,  65.7153,  81.8573,  39.5820,  42.5460,  14.6702,
        193.3005, 115.5951,  56.5382,  32.7032,  20.6026,  26.6044,  72.4638,
        366.6427, 287.7138,  29.5673, 497.4019,  21.7500,  96.6405,   6.6405,
        378.5478,  50.8278, 126.4862,  11.6234,  15.5325,  55.6529,  44.5503,
         33.6551, 114.6755,   8.6863,  16.6626,  22.5990, 105.2805, 172.6479,
         66.4041,  11.8986, 197.7281, 327.1071, 162.4853,  33.6933,  35.5338,
         20.7255, 172.5473,  34.6877,  58.8168,  97.9286,  17.1144,   8.1929,
         17.9060,  23.3008, 102.7108,  24.3877,  70.3678, 139.4349,   9.8268,
          6.1404,  29.5653,  54.3115, 188.3158,  13.8044,  28.5909,  25.0124,
         63.5011, 116.6791,  31.6912, 120.1187,  35.6465, 116.1934,  26.3773,
        442.5891,   7.2229,  81.6441,  58.5589,  49.7743, 272.7033, 106.6702,
         24.9417,  69.3891,  20.6295,  45.0643, 190.0102,  33.4722, 123.5978,
         27.8585,  35.3823,  13.4848,  24.5704,  38.2576,  94.49

tensor([ 59.9669,   9.2119, 396.1201, 136.6355,  55.4005,  62.2689,  37.6697,
        163.3080, 148.0048,  96.4116,  17.5775,  12.4617, 108.5077,  29.8978,
         41.8876, 170.3676,  34.6926,  14.7908,  95.9990,  92.4984,  23.7586,
        131.9558,  55.7615, 228.3190,  25.0191,  94.1251,  52.1510,  37.8341,
        237.6209,  45.0546,  18.5220, 177.7865, 140.3499, 122.1933,  45.1052,
         26.9741, 269.9996,  52.1672,  57.3574,  27.5533,  12.3990, 455.3328,
         39.2794,  22.1645,  42.2670,  48.4667, 161.8363,  25.1137,  29.7090,
         90.1683,  84.5994,  93.8575, 215.0044,  26.0234,  59.0993, 105.9388,
         21.3633,  21.5158, 151.1798,   7.5457,  35.8264,  41.4577, 131.4371,
        190.9295, 328.8539,  25.2513, 119.0039,  19.1278,  30.9047,  15.4061,
         38.0704,  95.0489, 220.1408,  23.0970,  55.5821,  39.0854, 318.0393,
         58.3189,  79.1648, 258.3784,  26.3643,  77.7344,  11.5396,  89.2016,
        185.4530,  51.7324, 101.9952,  64.4463,  77.4932, 114.81

tensor([ 27.3579, 210.9326,  51.8282,  44.8182,  24.6313,  14.5846,  43.0086,
        282.7767,  27.3265,  62.8494,  99.3385,  58.4021, 349.8619,  74.9614,
        101.5344, 309.8560,  64.7061,  46.6944,  38.9500, 195.4605, 228.6394,
         93.4997,  28.8875,  58.5711,  18.8713, 179.3668,  46.4215,  18.2881,
         28.7264,  15.8748,  23.0656, 113.3946,  28.4794, 112.5756,  23.5821,
        580.9188,  32.5946,  32.8106, 171.4842,  39.0452,  86.0601, 109.6088,
         36.0541, 177.7217,  43.2614, 116.8956,  34.9414,  99.9091, 265.3013,
        238.1642,  29.9430,  65.9202,  42.4110, 149.8705,  93.8546,  76.2430,
         34.2264,  69.9763,  27.1393,  24.1837, 111.3315,  24.1141,  33.5202,
        169.2002,  75.0522,  45.3195, 123.3743,  79.0591,  24.6380,  32.3486,
         16.7652, 106.3115,  31.1444,  89.7113,  73.2832, 206.7518,  85.6375,
         33.9735,  68.8983, 106.1284, 215.0960,  31.4347,  16.3344, 182.9655,
         36.4293,  31.0848, 177.5234, 145.8458,  28.5041, 600.18

tensor([ 77.4570,  83.0449,  46.2016,   8.8314,  26.8250,  42.5064, 173.6888,
         68.5943,  24.3186,  41.2402,  27.8670,  44.9586,  16.8868,  22.1048,
         13.4295, 151.3004,  23.7206, 112.7631, 300.9360,  36.9696, 108.8356,
         23.1857,  35.7146,  36.4524,  27.2478,   9.7352, 284.4702,  18.7018,
         51.4761,  49.5990,  41.3453, 135.0311,  75.8121, 103.7297,  21.9392,
         18.6782, 166.8221, 106.2686,  43.6649, 241.3504, 115.8188, 183.5425,
         18.9900,  46.8522, 245.8566,  30.1701,  45.7556, 131.1743, 312.5179,
         24.8713,  22.5070, 137.4194, 233.0807,  20.5516,  91.8336,  35.2946,
        161.4788,  14.5633,  37.2938,  27.0872,  69.1223,  22.3171, 118.9724,
         83.4392,  24.2213, 113.4885,  54.5268,  18.9996,  33.4200,  65.8647,
         14.7209,  45.5539,  81.5469, 108.6241, 158.4895,  15.7887,  62.0998,
         12.0018,  20.6418,  47.8108,  22.8398,  26.9850,  58.7007, 128.8454,
         17.7980,  34.7259,   8.4123, 154.2922,  19.6840,  12.31

tensor([ 31.7081,  18.6831,  78.8398,  10.6590,  46.3483,  18.3341,  19.8141,
         88.4615,  40.4108, 101.8274,  13.2084,  22.2253,  14.1689,  24.3449,
         74.3339,  12.4206,  25.2249,  32.2098,  39.6486,  46.3315,  31.0006,
         35.3125, 121.0542,  49.7255,  71.6052, 101.4831,  81.9860,  75.1212,
         33.6295,  28.7951,  63.7510, 110.0280,  29.4556,  91.0118,  55.6286,
        143.9498,  73.4168,  79.7388,  55.5174,  10.8446,  43.4517,  76.6804,
         85.3461, 250.6389, 144.8897,  50.3449, 173.1325,  13.5330, 114.5647,
         13.4807, 208.7364,  53.9773, 218.5154,  19.1821,  20.8029,  24.6444,
        138.0505,  65.7161, 191.3309,  43.3450,  83.0709,  64.6204,  16.3602,
        113.0676,  21.0281, 142.5901,  13.9551,  34.2753, 170.4828,  80.2406,
        134.8135,  23.8019, 129.4702,  45.9262, 267.6263,  16.0074,  39.7853,
         18.5971,  17.9919, 259.1789, 349.7615,  56.7514,  28.1970,  22.3561,
        181.3813,  44.5015,  14.0744,  51.0220,  85.0257,  19.10

...

tensor([ 66.4852,  73.2652,  80.5930,  63.3315,  82.6400,  82.6550,  90.5392,
         14.1949,  43.4549,  68.8357,  38.0536,  76.5392,  32.1259,  76.1832,
         30.4117, 141.9323, 123.9281,  31.1896,   9.1284, 105.3585, 160.0286,
         69.0499,  48.2546,   5.5838,   9.4513, 159.5209,  32.9498,  48.4514,
         36.2661,  23.8757,  41.0800,  27.9068,  19.8419,  32.5743,  20.6043,
         69.2343,  18.7141,  38.1970,  91.1789,  38.4597,  25.2933, 102.4654,
         15.6062,  27.4808,  79.1355,  50.6948, 203.8867,  14.8571,  52.6532,
         16.8226,  86.7326,  77.0840,  31.9685,  71.3769,  30.1264,  51.5385,
        168.5518,  13.1219,  14.6209, 186.1901,  38.5119,  61.3286,  54.7493,
         54.5255,  52.3545,  70.2866,  96.3731,  44.7762,  79.9790,  89.2949,
         24.3707,  11.8781,  23.8375,  43.0915,  22.8047,  21.5869,  20.4238,
         91.8030,  48.9255, 245.0200,  35.9766,  22.8822,  28.8193,  51.5511,
        133.1364,  58.6187,  72.5093, 106.4989,  89.5721,  54.19

tensor([207.7793,  37.9687,  92.6993,  49.0449,  26.3941, 110.7983,  42.6238,
         17.4914,  10.5087,  29.6992,  64.5622,  29.3176,  30.1098, 144.6065,
         67.3060,  36.9077, 150.4455,  43.2993,  23.6387,  55.0622,  21.8157,
         28.0196,  26.2924,  42.4895,  54.8088,  62.4685,  52.2847, 127.0322,
         15.6720,  17.9725, 110.2190,  79.6105, 159.8601,  43.7984,  11.8108,
         65.9383,  44.5277, 230.2999,  73.5781,  78.5354,  10.2789,  12.3408,
        123.6986,  61.9494,  50.7919,  21.1373, 197.1941,  63.5889,  24.6796,
         59.9781,  25.7355,  49.5463,  22.0757,  58.8829,  41.2793, 161.9796,
         17.8588,  88.5753, 193.7728, 131.5060,  13.8132,  14.9877,  72.3066,
         82.5423,  23.4823,  14.8317, 297.3674, 126.3236,  27.9847,  13.3292,
        129.2988,  31.3680, 131.1676,  69.2791,  85.7290, 161.7613,  87.2326,
         65.3505,  50.2274, 221.6565,  17.3548,  30.2670, 138.6572,  23.8608,
         70.6559,  15.5152,  26.8182,  59.1996,  90.0030,  41.09

tensor([ 55.1756,  82.3140, 113.8604,  30.1269,  15.0897, 109.8294,  46.1077,
         17.3454,  88.1145, 111.9029,  50.5576,  28.0969,  14.6297,  35.1695,
         30.4423,  21.4298,  26.7662,  42.4525, 211.4887,  25.3471,  16.8052,
         99.7463, 168.6556,  57.3413,  70.9248, 174.8118,  10.6302,  15.3891,
         64.9514,  12.9221,  72.5785,   7.9959,  17.4937,  20.5974,   8.0955,
         27.9006,  55.5339,  32.1577,  47.9673,  93.2209,  22.1527,  24.1329,
         30.3567,  23.0540,  47.5323,  18.7745,  24.3195, 149.0462,  23.4638,
        152.7853, 100.3891,  77.9044, 125.7392,  61.0245,  29.3353,  17.3670,
         61.2215,  29.6639,  24.3167, 128.6917, 126.7151,  20.9448,  14.9323,
         21.8155,  27.5986, 170.7095,  37.9934, 118.3204,  65.2578,  19.6366,
         81.7956,  47.4552,  67.6813, 115.4147,  52.8745,  73.1820, 108.3172,
          5.9102, 134.4963,  21.3317, 186.2573, 175.0479,  44.7876,  73.2718,
         83.4645,  87.0481,  15.2496, 216.9420,  15.0865,  18.63

tensor([ 83.1525,  39.3290,  89.5147,  77.0433,  22.1276,  14.7286,  36.2106,
         22.4638,  10.8099,  31.1688, 150.2072,  71.6228,  41.2517,  44.1277,
         22.9280,   7.6484,  35.0081,  11.5545,  33.5960,  33.6418,  72.7657,
         24.4091,  34.8869,  86.2157,  59.0606,  57.5687, 138.5396,  97.9251,
         58.0571,  54.3511, 115.8921, 184.2714,  66.4825, 216.4568,  19.1235,
          5.2480,  61.7100,  35.0134,  14.3765, 141.7836,  65.8639, 138.4902,
         81.1476, 162.5502, 264.9859,  77.4110,  77.8266,  16.9838,  41.2757,
         24.8088,  13.8895,  74.0809,  52.8996,  30.6800,  11.5787,  17.5191,
         36.1066,  10.3610,  27.7859,   9.0549,  21.6615,  24.1262,  12.9075,
         26.8930, 159.9944,  18.6260,  74.9965,  15.9014,  43.2893,  73.3492,
         67.7907,  17.2197,  20.9746, 115.1467,  21.1385,  25.2532, 122.6641,
         33.2495, 116.9455,  90.9244,  29.4788,  88.8946,  30.0222,  37.0316,
         84.8059,  69.0031,  31.8376, 164.7195,  73.3339,  19.54

tensor([  9.5293,  41.4462, 133.5109,  38.7438,  91.9010,  30.9583,  19.4073,
        234.9658,  33.7486,  32.5353,  75.1034,  21.3079,  13.0784,  15.8783,
         31.9973,  26.1363,  99.1944,  47.1929,  80.7571,  58.6143, 186.7939,
         18.9430,  12.2516,  45.2752,  23.7239,  74.3860,  18.5801, 142.2322,
        104.0914, 165.7477,  21.1872,   9.9267,  29.0132,  16.9213,   9.2140,
         15.6385,  45.5239,  47.6594,  42.1188,  23.2342,  34.3763, 177.9339,
         12.5089,  17.5543,  55.1913,  80.4915,  58.7098,  24.2996,  15.1843,
         16.1385,  20.1830, 111.2806,  14.6715,  30.1397, 142.1276,  67.9336,
         94.6794,   7.7505,  43.5028,  39.1919,  20.7348,  20.8197,  37.0312,
        196.4277, 178.5278,  85.1437,  60.8750,  11.1624,  51.2856,  68.1519,
         22.5987,  27.6645,  14.6818,  17.5218,  89.0921, 127.9376,  19.2780,
         19.0803, 116.7307, 168.4431,  49.5923,  40.2006,  84.4826, 101.4225,
        177.3055, 206.5438,  94.7486,  80.2169,  12.6342,  62.95

In [22]:
leave_k_eval(model, tr, val, leavek=1, K=5)

{'avghit': 0.6173351010938017,
 'precision': 0.6173351010938017,
 'map': 0.40631698154900026,
 'ndcg': 0.45879305542103144}

In [42]:
qed = []
import models.ae
importlib.reload(models.ae)
model = models.ae.MultiVAE(best_m['dim'] + [n_items], dropout=best_m['dropout'])
importlib.reload(models.ae)
import neuralsort.neuralobjs

use_vae=True
loader = DataLoader(ae_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
model = models.ae.MultiVAE(best_m['dim'] + [n_items], dropout=best_m['dropout'])
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=best_m['lr'])
lm = -1
update_count = 0
tr_t = {
    'l': [],
    ('precision', 5):[],
    ('precision', 10):[],
    ('map', 5): [],
    ('map', 10): [],
    ('ndcg', 5): [],
    ('ndcg', 10): [] ,
}
val_t = {
    'l': [],
    ('precision', 5):[],
    ('precision', 10):[],
    ('map', 5): [],
    ('map', 10): [],
    ('ndcg', 5): [],
    ('ndcg', 10): [] ,
}
te_t = {
    'l': [],
    ('precision', 5):[],
    ('precision', 10):[],
    ('map', 5): [],
    ('map', 10): [],
    ('ndcg', 5): [],
    ('ndcg', 10): [] ,
}
for epoch in (range(1, 1000 + 1)):
    model = model.train()
    model.training = True
    tr_losses = []
    # train for one epoch
    for uid, rowl in (loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        loss = None 
        scores, mu, logvar = model.forward(row)
        if total_anneal_steps > 0:
            anneal_cap = min(_anneal_cap, 1. * update_count / total_anneal_steps)
        else:
            anneal_cap = _anneal_cap
        loss  = models.ae.loss_function(scores, row, mu, logvar)
        update_count += 1
        (loss.mean()).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        tr_losses.append(loss.detach().unsqueeze(-1)) 
        optimizer.step()
    
    if (epoch % 20 == 0):
        model.eval()
        model.training=False
        tr_loss = float(torch.cat(tr_losses).mean().detach().cpu().numpy())
        vad_scores, mu, logvar = model.forward(vin_dense)
        vad_loss = models.ae.loss_function(vad_scores, vin_dense, mu, logvar, anneal_cap)
        vad_loss = float(vad_loss.mean().detach().cpu().numpy())
        te_scores, mu, logvar = model.forward(tin_dense)
        te_loss = models.ae.loss_function(te_scores, tin_dense, mu, logvar, anneal_cap)
        te_loss = float(te_loss.mean().detach().cpu().numpy())

        tr_t['l'].append(tr_loss)
        val_t['l'].append(vad_loss)
        te_t['l'].append(te_loss)
        wrapper = models.ae.implicitWrapper(model, naive_sparse2tensor, vae=True)
        ret = (tr_loss, vad_loss)

        for store, q, a in  zip([tr_t, val_t, te_t], [trin, vin, tin], [trte, vo, to]):
            for topk in [5, 10]:
                scs = ranking_metrics_at_k(wrapper, q, a, K=topk, num_threads=4)
                for ty in ['map', 'precision', 'ndcg']:
                    store[(ty, topk)].append(scs[ty])
            print(store)
        qed.append(ret)
no_add = qed

{'l': [289.5909423828125], ('precision', 5): [0.11661874640597812], ('precision', 10): [0.09051178838412861], ('map', 5): [0.08760877899175729], ('map', 10): [0.07431090429697304], ('ndcg', 5): [0.1408258883110016], ('ndcg', 10): [0.14093437733598943]}
{'l': [227.49560546875], ('precision', 5): [0.10240000000000052], ('precision', 10): [0.0801], ('map', 5): [0.07979027777777789], ('map', 10): [0.06817714962836993], ('ndcg', 5): [0.12668102304479953], ('ndcg', 10): [0.12857793567911105]}
{'l': [217.5330352783203], ('precision', 5): [0.09760000000000048], ('precision', 10): [0.07919999999999996], ('map', 5): [0.08259361111111112], ('map', 10): [0.07535227040816322], ('ndcg', 5): [0.12888342095491465], ('ndcg', 10): [0.1372432495542677]}
{'l': [289.5909423828125, 282.6343994140625], ('precision', 5): [0.11661874640597812, 0.14249568717653538], ('precision', 10): [0.09051178838412861, 0.11000575043128286], ('map', 5): [0.08760877899175729, 0.11359409941856713], ('map', 10): [0.074310904296

{'l': [227.49560546875, 219.5446014404297, 214.9517822265625, 212.13624572753906, 210.46603393554688, 209.47317504882812], ('precision', 5): [0.10240000000000052, 0.12940000000000063, 0.13860000000000033, 0.14480000000000023, 0.15060000000000018, 0.15300000000000008], ('precision', 10): [0.0801, 0.09879999999999979, 0.11279999999999975, 0.11819999999999958, 0.12269999999999964, 0.12609999999999957], ('map', 5): [0.07979027777777789, 0.10221972222222232, 0.11133083333333343, 0.11578083333333335, 0.12177249999999999, 0.12651111111111119], ('map', 10): [0.06817714962836993, 0.08694477812421274, 0.0984474332325523, 0.10310245354623332, 0.1084870249118166, 0.11340036029226509], ('ndcg', 5): [0.12668102304479953, 0.16003499257847076, 0.1722412752772343, 0.17852601156579612, 0.18584322767898317, 0.19150194941408372], ('ndcg', 10): [0.12857793567911105, 0.15894725213487954, 0.1775323667095789, 0.18577447167452246, 0.19406701399502402, 0.20127833998610503]}
{'l': [217.5330352783203, 209.7557983

{'l': [289.5909423828125, 282.6343994140625, 276.35833740234375, 273.54266357421875, 270.0124206542969, 270.3948669433594, 269.46160888671875, 268.6343994140625, 267.2833557128906], ('precision', 5): [0.11661874640597812, 0.14249568717653538, 0.15606670500287304, 0.16152961472110233, 0.1679125934445072, 0.1728004600345016, 0.17642323174238014, 0.1809085681426103, 0.18355376653248948], ('precision', 10): [0.09051178838412861, 0.11000575043128286, 0.1218228867165047, 0.12829212190914435, 0.13326624496837391, 0.1369177688326638, 0.1396492236917782, 0.14229442208165735, 0.14468085106383097], ('map', 5): [0.08760877899175729, 0.11359409941856713, 0.1270865280173789, 0.132915867995655, 0.1382136924158199, 0.14260821992204956, 0.14691712989585304, 0.14990216280109875, 0.1529458181585839], ('map', 10): [0.07431090429697304, 0.0961155461348596, 0.10852501926590045, 0.11496247305707767, 0.1199610208250658, 0.12392526332780882, 0.12764893792692641, 0.13034509945065914, 0.1330272191246099], ('ndcg

{'l': [289.5909423828125, 282.6343994140625, 276.35833740234375, 273.54266357421875, 270.0124206542969, 270.3948669433594, 269.46160888671875, 268.6343994140625, 267.2833557128906, 267.1373596191406, 265.5999450683594], ('precision', 5): [0.11661874640597812, 0.14249568717653538, 0.15606670500287304, 0.16152961472110233, 0.1679125934445072, 0.1728004600345016, 0.17642323174238014, 0.1809085681426103, 0.18355376653248948, 0.18430132259919457, 0.18752156411730883], ('precision', 10): [0.09051178838412861, 0.11000575043128286, 0.1218228867165047, 0.12829212190914435, 0.13326624496837391, 0.1369177688326638, 0.1396492236917782, 0.14229442208165735, 0.14468085106383097, 0.14695227142035797, 0.1487636572742972], ('map', 5): [0.08760877899175729, 0.11359409941856713, 0.1270865280173789, 0.132915867995655, 0.1382136924158199, 0.14260821992204956, 0.14691712989585304, 0.14990216280109875, 0.1529458181585839, 0.15342446169573817, 0.15649750814644423], ('map', 10): [0.07431090429697304, 0.0961155

{'l': [217.5330352783203, 209.75579833984375, 205.34423828125, 202.57273864746094, 200.94711303710938, 199.9864959716797, 199.13650512695312, 198.41632080078125, 197.9143524169922, 197.4642333984375, 197.07662963867188, 196.75135803222656], ('precision', 5): [0.09760000000000048, 0.12180000000000067, 0.13320000000000054, 0.14160000000000036, 0.14600000000000032, 0.15040000000000017, 0.1526000000000001, 0.1564000000000001, 0.15660000000000004, 0.158, 0.159, 0.1599999999999999], ('precision', 10): [0.07919999999999996, 0.09629999999999969, 0.10849999999999968, 0.11379999999999957, 0.11759999999999954, 0.11989999999999966, 0.12109999999999968, 0.12259999999999965, 0.12469999999999955, 0.1250999999999996, 0.12669999999999956, 0.12629999999999958], ('map', 5): [0.08259361111111112, 0.10588777777777775, 0.11556694444444443, 0.1210824999999999, 0.12333111111111102, 0.1279930555555555, 0.1303272222222221, 0.13357944444444442, 0.13526888888888877, 0.13585555555555548, 0.13614472222222215, 0.137

...

{'l': [217.5330352783203, 209.75579833984375, 205.34423828125, 202.57273864746094, 200.94711303710938, 199.9864959716797, 199.13650512695312, 198.41632080078125, 197.9143524169922, 197.4642333984375, 197.07662963867188, 196.75135803222656, 196.50357055664062, 196.24407958984375, 196.05755615234375, 195.85609436035156, 195.65623474121094, 195.54336547851562, 195.3961181640625, 195.27359008789062, 195.1609344482422, 195.06124877929688, 195.02476501464844, 194.98326110839844, 194.82798767089844, 194.86610412597656, 194.71986389160156], ('precision', 5): [0.09760000000000048, 0.12180000000000067, 0.13320000000000054, 0.14160000000000036, 0.14600000000000032, 0.15040000000000017, 0.1526000000000001, 0.1564000000000001, 0.15660000000000004, 0.158, 0.159, 0.1599999999999999, 0.1595999999999999, 0.1577999999999999, 0.15979999999999991, 0.1599999999999999, 0.16099999999999984, 0.16139999999999988, 0.16299999999999987, 0.16139999999999988, 0.16119999999999987, 0.16099999999999992, 0.160799999999

{'l': [217.5330352783203, 209.75579833984375, 205.34423828125, 202.57273864746094, 200.94711303710938, 199.9864959716797, 199.13650512695312, 198.41632080078125, 197.9143524169922, 197.4642333984375, 197.07662963867188, 196.75135803222656, 196.50357055664062, 196.24407958984375, 196.05755615234375, 195.85609436035156, 195.65623474121094, 195.54336547851562, 195.3961181640625, 195.27359008789062, 195.1609344482422, 195.06124877929688, 195.02476501464844, 194.98326110839844, 194.82798767089844, 194.86610412597656, 194.71986389160156, 194.7155303955078], ('precision', 5): [0.09760000000000048, 0.12180000000000067, 0.13320000000000054, 0.14160000000000036, 0.14600000000000032, 0.15040000000000017, 0.1526000000000001, 0.1564000000000001, 0.15660000000000004, 0.158, 0.159, 0.1599999999999999, 0.1595999999999999, 0.1577999999999999, 0.15979999999999991, 0.1599999999999999, 0.16099999999999984, 0.16139999999999988, 0.16299999999999987, 0.16139999999999988, 0.16119999999999987, 0.16099999999999

{'l': [227.49560546875, 219.5446014404297, 214.9517822265625, 212.13624572753906, 210.46603393554688, 209.47317504882812, 208.61630249023438, 207.87106323242188, 207.35023498535156, 206.86973571777344, 206.470703125, 206.1251220703125, 205.86764526367188, 205.58001708984375, 205.3716583251953, 205.17185974121094, 204.96006774902344, 204.8592529296875, 204.688720703125, 204.54920959472656, 204.43365478515625, 204.29794311523438, 204.27822875976562, 204.2465057373047, 204.07785034179688, 204.11155700683594, 203.9482879638672, 203.95773315429688, 203.96775817871094], ('precision', 5): [0.10240000000000052, 0.12940000000000063, 0.13860000000000033, 0.14480000000000023, 0.15060000000000018, 0.15300000000000008, 0.15719999999999987, 0.15939999999999985, 0.16139999999999968, 0.16419999999999965, 0.1647999999999996, 0.16619999999999963, 0.16659999999999967, 0.1663999999999996, 0.16659999999999964, 0.16699999999999962, 0.16619999999999963, 0.16819999999999968, 0.1673999999999996, 0.166199999999

{'l': [289.5909423828125, 282.6343994140625, 276.35833740234375, 273.54266357421875, 270.0124206542969, 270.3948669433594, 269.46160888671875, 268.6343994140625, 267.2833557128906, 267.1373596191406, 265.5999450683594, 265.80865478515625, 265.8110656738281, 265.00244140625, 265.22613525390625, 262.9532165527344, 262.597412109375, 263.26336669921875, 262.4729309082031, 263.3291320800781, 262.25579833984375, 260.999755859375, 260.1268005371094, 260.2434997558594, 261.2622375488281, 260.0489807128906, 259.48114013671875, 259.8913269042969, 259.4231872558594, 259.11285400390625], ('precision', 5): [0.11661874640597812, 0.14249568717653538, 0.15606670500287304, 0.16152961472110233, 0.1679125934445072, 0.1728004600345016, 0.17642323174238014, 0.1809085681426103, 0.18355376653248948, 0.18430132259919457, 0.18752156411730883, 0.1887291546866016, 0.1913168487636575, 0.1947096032202418, 0.19689476710753342, 0.19844738355376726, 0.2002300172512947, 0.2013801035077637, 0.20327774583093805, 0.20494

{'l': [217.5330352783203, 209.75579833984375, 205.34423828125, 202.57273864746094, 200.94711303710938, 199.9864959716797, 199.13650512695312, 198.41632080078125, 197.9143524169922, 197.4642333984375, 197.07662963867188, 196.75135803222656, 196.50357055664062, 196.24407958984375, 196.05755615234375, 195.85609436035156, 195.65623474121094, 195.54336547851562, 195.3961181640625, 195.27359008789062, 195.1609344482422, 195.06124877929688, 195.02476501464844, 194.98326110839844, 194.82798767089844, 194.86610412597656, 194.71986389160156, 194.7155303955078, 194.7046661376953, 194.66481018066406], ('precision', 5): [0.09760000000000048, 0.12180000000000067, 0.13320000000000054, 0.14160000000000036, 0.14600000000000032, 0.15040000000000017, 0.1526000000000001, 0.1564000000000001, 0.15660000000000004, 0.158, 0.159, 0.1599999999999999, 0.1595999999999999, 0.1577999999999999, 0.15979999999999991, 0.1599999999999999, 0.16099999999999984, 0.16139999999999988, 0.16299999999999987, 0.16139999999999988

In [43]:
for epoch in (range(1, 400 + 1)):
    model = model.train()
    model.training = True
    tr_losses = []
    # train for one epoch
    for uid, rowl in (loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        loss = None 
        scores, mu, logvar = model.forward(row)
        if total_anneal_steps > 0:
            anneal_cap = min(_anneal_cap, 1. * update_count / total_anneal_steps)
        else:
            anneal_cap = _anneal_cap
        loss  = models.ae.loss_function(scores, row, mu, logvar)
        update_count += 1
        (loss.mean()).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        tr_losses.append(loss.detach().unsqueeze(-1)) 
        optimizer.step()
    
    if (epoch % 20 == 0):
        model.eval()
        model.training=False
        tr_loss = float(torch.cat(tr_losses).mean().detach().cpu().numpy())
        vad_scores, mu, logvar = model.forward(vin_dense)
        vad_loss = models.ae.loss_function(vad_scores, vin_dense, mu, logvar, anneal_cap)
        vad_loss = float(vad_loss.mean().detach().cpu().numpy())
        te_scores, mu, logvar = model.forward(tin_dense)
        te_loss = models.ae.loss_function(te_scores, tin_dense, mu, logvar, anneal_cap)
        te_loss = float(te_loss.mean().detach().cpu().numpy())

        tr_t['l'].append(tr_loss)
        val_t['l'].append(vad_loss)
        te_t['l'].append(te_loss)
        wrapper = models.ae.implicitWrapper(model, naive_sparse2tensor, vae=True)
        ret = (tr_loss, vad_loss)

        for store, q, a in  zip([tr_t, val_t, te_t], [trin, vin, tin], [trte, vo, to]):
            for topk in [5, 10]:
                scs = ranking_metrics_at_k(wrapper, q, a, K=topk, num_threads=4)
                for ty in ['map', 'precision', 'ndcg']:
                    store[(ty, topk)].append(scs[ty])
            print(store)
        qed.append(ret)
no_add = qed

{'l': [289.5909423828125, 282.6343994140625, 276.35833740234375, 273.54266357421875, 270.0124206542969, 270.3948669433594, 269.46160888671875, 268.6343994140625, 267.2833557128906, 267.1373596191406, 265.5999450683594, 265.80865478515625, 265.8110656738281, 265.00244140625, 265.22613525390625, 262.9532165527344, 262.597412109375, 263.26336669921875, 262.4729309082031, 263.3291320800781, 262.25579833984375, 260.999755859375, 260.1268005371094, 260.2434997558594, 261.2622375488281, 260.0489807128906, 259.48114013671875, 259.8913269042969, 259.4231872558594, 259.11285400390625, 259.15362548828125], ('precision', 5): [0.11661874640597812, 0.14249568717653538, 0.15606670500287304, 0.16152961472110233, 0.1679125934445072, 0.1728004600345016, 0.17642323174238014, 0.1809085681426103, 0.18355376653248948, 0.18430132259919457, 0.18752156411730883, 0.1887291546866016, 0.1913168487636575, 0.1947096032202418, 0.19689476710753342, 0.19844738355376726, 0.2002300172512947, 0.2013801035077637, 0.203277

{'l': [217.5330352783203, 209.75579833984375, 205.34423828125, 202.57273864746094, 200.94711303710938, 199.9864959716797, 199.13650512695312, 198.41632080078125, 197.9143524169922, 197.4642333984375, 197.07662963867188, 196.75135803222656, 196.50357055664062, 196.24407958984375, 196.05755615234375, 195.85609436035156, 195.65623474121094, 195.54336547851562, 195.3961181640625, 195.27359008789062, 195.1609344482422, 195.06124877929688, 195.02476501464844, 194.98326110839844, 194.82798767089844, 194.86610412597656, 194.71986389160156, 194.7155303955078, 194.7046661376953, 194.66481018066406, 194.5948486328125], ('precision', 5): [0.09760000000000048, 0.12180000000000067, 0.13320000000000054, 0.14160000000000036, 0.14600000000000032, 0.15040000000000017, 0.1526000000000001, 0.1564000000000001, 0.15660000000000004, 0.158, 0.159, 0.1599999999999999, 0.1595999999999999, 0.1577999999999999, 0.15979999999999991, 0.1599999999999999, 0.16099999999999984, 0.16139999999999988, 0.16299999999999987, 

{'l': [227.49560546875, 219.5446014404297, 214.9517822265625, 212.13624572753906, 210.46603393554688, 209.47317504882812, 208.61630249023438, 207.87106323242188, 207.35023498535156, 206.86973571777344, 206.470703125, 206.1251220703125, 205.86764526367188, 205.58001708984375, 205.3716583251953, 205.17185974121094, 204.96006774902344, 204.8592529296875, 204.688720703125, 204.54920959472656, 204.43365478515625, 204.29794311523438, 204.27822875976562, 204.2465057373047, 204.07785034179688, 204.11155700683594, 203.9482879638672, 203.95773315429688, 203.96775817871094, 203.90040588378906, 203.8463897705078, 203.96859741210938], ('precision', 5): [0.10240000000000052, 0.12940000000000063, 0.13860000000000033, 0.14480000000000023, 0.15060000000000018, 0.15300000000000008, 0.15719999999999987, 0.15939999999999985, 0.16139999999999968, 0.16419999999999965, 0.1647999999999996, 0.16619999999999963, 0.16659999999999967, 0.1663999999999996, 0.16659999999999964, 0.16699999999999962, 0.166199999999999

{'l': [289.5909423828125, 282.6343994140625, 276.35833740234375, 273.54266357421875, 270.0124206542969, 270.3948669433594, 269.46160888671875, 268.6343994140625, 267.2833557128906, 267.1373596191406, 265.5999450683594, 265.80865478515625, 265.8110656738281, 265.00244140625, 265.22613525390625, 262.9532165527344, 262.597412109375, 263.26336669921875, 262.4729309082031, 263.3291320800781, 262.25579833984375, 260.999755859375, 260.1268005371094, 260.2434997558594, 261.2622375488281, 260.0489807128906, 259.48114013671875, 259.8913269042969, 259.4231872558594, 259.11285400390625, 259.15362548828125, 259.3638916015625, 258.65191650390625], ('precision', 5): [0.11661874640597812, 0.14249568717653538, 0.15606670500287304, 0.16152961472110233, 0.1679125934445072, 0.1728004600345016, 0.17642323174238014, 0.1809085681426103, 0.18355376653248948, 0.18430132259919457, 0.18752156411730883, 0.1887291546866016, 0.1913168487636575, 0.1947096032202418, 0.19689476710753342, 0.19844738355376726, 0.2002300

{'l': [217.5330352783203, 209.75579833984375, 205.34423828125, 202.57273864746094, 200.94711303710938, 199.9864959716797, 199.13650512695312, 198.41632080078125, 197.9143524169922, 197.4642333984375, 197.07662963867188, 196.75135803222656, 196.50357055664062, 196.24407958984375, 196.05755615234375, 195.85609436035156, 195.65623474121094, 195.54336547851562, 195.3961181640625, 195.27359008789062, 195.1609344482422, 195.06124877929688, 195.02476501464844, 194.98326110839844, 194.82798767089844, 194.86610412597656, 194.71986389160156, 194.7155303955078, 194.7046661376953, 194.66481018066406, 194.5948486328125, 194.69700622558594, 194.78758239746094], ('precision', 5): [0.09760000000000048, 0.12180000000000067, 0.13320000000000054, 0.14160000000000036, 0.14600000000000032, 0.15040000000000017, 0.1526000000000001, 0.1564000000000001, 0.15660000000000004, 0.158, 0.159, 0.1599999999999999, 0.1595999999999999, 0.1577999999999999, 0.15979999999999991, 0.1599999999999999, 0.16099999999999984, 0.

...

{'l': [217.5330352783203, 209.75579833984375, 205.34423828125, 202.57273864746094, 200.94711303710938, 199.9864959716797, 199.13650512695312, 198.41632080078125, 197.9143524169922, 197.4642333984375, 197.07662963867188, 196.75135803222656, 196.50357055664062, 196.24407958984375, 196.05755615234375, 195.85609436035156, 195.65623474121094, 195.54336547851562, 195.3961181640625, 195.27359008789062, 195.1609344482422, 195.06124877929688, 195.02476501464844, 194.98326110839844, 194.82798767089844, 194.86610412597656, 194.71986389160156, 194.7155303955078, 194.7046661376953, 194.66481018066406, 194.5948486328125, 194.69700622558594, 194.78758239746094, 194.6175079345703, 194.4956512451172, 194.40553283691406, 194.2350311279297, 194.22576904296875, 194.03977966308594, 193.91635131835938, 193.87876892089844, 193.68028259277344, 193.61415100097656, 193.49732971191406, 193.46051025390625, 193.30300903320312, 193.23477172851562], ('precision', 5): [0.09760000000000048, 0.12180000000000067, 0.1332

{'l': [227.49560546875, 219.5446014404297, 214.9517822265625, 212.13624572753906, 210.46603393554688, 209.47317504882812, 208.61630249023438, 207.87106323242188, 207.35023498535156, 206.86973571777344, 206.470703125, 206.1251220703125, 205.86764526367188, 205.58001708984375, 205.3716583251953, 205.17185974121094, 204.96006774902344, 204.8592529296875, 204.688720703125, 204.54920959472656, 204.43365478515625, 204.29794311523438, 204.27822875976562, 204.2465057373047, 204.07785034179688, 204.11155700683594, 203.9482879638672, 203.95773315429688, 203.96775817871094, 203.90040588378906, 203.8463897705078, 203.96859741210938, 204.056640625, 203.8594207763672, 203.73776245117188, 203.67031860351562, 203.4450225830078, 203.45870971679688, 203.27615356445312, 203.1498565673828, 203.10812377929688, 202.8893280029297, 202.84896850585938, 202.72108459472656, 202.71632385253906, 202.55117797851562, 202.4641571044922, 202.41143798828125], ('precision', 5): [0.10240000000000052, 0.12940000000000063,

{'l': [289.5909423828125, 282.6343994140625, 276.35833740234375, 273.54266357421875, 270.0124206542969, 270.3948669433594, 269.46160888671875, 268.6343994140625, 267.2833557128906, 267.1373596191406, 265.5999450683594, 265.80865478515625, 265.8110656738281, 265.00244140625, 265.22613525390625, 262.9532165527344, 262.597412109375, 263.26336669921875, 262.4729309082031, 263.3291320800781, 262.25579833984375, 260.999755859375, 260.1268005371094, 260.2434997558594, 261.2622375488281, 260.0489807128906, 259.48114013671875, 259.8913269042969, 259.4231872558594, 259.11285400390625, 259.15362548828125, 259.3638916015625, 258.65191650390625, 258.1945495605469, 258.20001220703125, 259.23175048828125, 257.7560729980469, 257.4389953613281, 256.48321533203125, 257.77520751953125, 256.35614013671875, 256.6065368652344, 257.4552917480469, 256.27496337890625, 256.626953125, 256.97039794921875, 256.0778503417969, 256.2647705078125, 256.3413391113281], ('precision', 5): [0.11661874640597812, 0.142495687

{'l': [217.5330352783203, 209.75579833984375, 205.34423828125, 202.57273864746094, 200.94711303710938, 199.9864959716797, 199.13650512695312, 198.41632080078125, 197.9143524169922, 197.4642333984375, 197.07662963867188, 196.75135803222656, 196.50357055664062, 196.24407958984375, 196.05755615234375, 195.85609436035156, 195.65623474121094, 195.54336547851562, 195.3961181640625, 195.27359008789062, 195.1609344482422, 195.06124877929688, 195.02476501464844, 194.98326110839844, 194.82798767089844, 194.86610412597656, 194.71986389160156, 194.7155303955078, 194.7046661376953, 194.66481018066406, 194.5948486328125, 194.69700622558594, 194.78758239746094, 194.6175079345703, 194.4956512451172, 194.40553283691406, 194.2350311279297, 194.22576904296875, 194.03977966308594, 193.91635131835938, 193.87876892089844, 193.68028259277344, 193.61415100097656, 193.49732971191406, 193.46051025390625, 193.30300903320312, 193.23477172851562, 193.17562866210938, 193.17103576660156], ('precision', 5): [0.097600

{'l': [227.49560546875, 219.5446014404297, 214.9517822265625, 212.13624572753906, 210.46603393554688, 209.47317504882812, 208.61630249023438, 207.87106323242188, 207.35023498535156, 206.86973571777344, 206.470703125, 206.1251220703125, 205.86764526367188, 205.58001708984375, 205.3716583251953, 205.17185974121094, 204.96006774902344, 204.8592529296875, 204.688720703125, 204.54920959472656, 204.43365478515625, 204.29794311523438, 204.27822875976562, 204.2465057373047, 204.07785034179688, 204.11155700683594, 203.9482879638672, 203.95773315429688, 203.96775817871094, 203.90040588378906, 203.8463897705078, 203.96859741210938, 204.056640625, 203.8594207763672, 203.73776245117188, 203.67031860351562, 203.4450225830078, 203.45870971679688, 203.27615356445312, 203.1498565673828, 203.10812377929688, 202.8893280029297, 202.84896850585938, 202.72108459472656, 202.71632385253906, 202.55117797851562, 202.4641571044922, 202.41143798828125, 202.42738342285156, 202.3318634033203], ('precision', 5): [0.

In [13]:
qed = []
import models.ae
importlib.reload(models.ae)
import neuralsort.neuralobjs
importlib.reload(models.ae)
use_vae=True
loader = DataLoader(ae_dataset, batch_size=batch_size, shuffle=True, num_workers=6)
if use_vae:
    model = models.ae.MultiVAE([200] + [n_items], dropout=0.5)
else:
    model = models.ae.MultiDAE([200] + [n_items], dropout=0.5)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=5 * 1e-4)
sc = neuralsort.neuralobjs.SC()
lm = -1 
tr_tt = {
    5: [],
    10: []
}
val_tt = {
    5: [],
    10: [],
}
lolis = []
update_count = 0
for epoch in (range(1, 1000 + 1)):
    model = model.train()
    model.training = True
    tr_losses = []
    loss2s =[] 
    # train for one epoch
    for uid, rowl in (loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        loss = None 
        if use_vae:
            scores, mu, logvar = model.forward(row)
            if total_anneal_steps > 0:
                anneal_cap = min(anneal_cap, 1. * update_count / total_anneal_steps)
            else:
                anneal = anneal_cap
            loss  = models.ae.loss_function(scores, row, mu, logvar)
            update_count += 1

        else:
            scores = model.forward(row)
            loss  = models.loss.MultinomialLoss(row, scores)

        loss2 = neuMapLoss(sc, scores, row, topk=300, k=5, tau=15.0, use_top=True)
        loss2s.append(loss2.unsqueeze(-1))
        (loss.mean() + 10.0 * loss2.mean()).backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        tr_losses.append(loss.detach().unsqueeze(-1)) 
        optimizer.step()
    
    if (epoch % 10 == 0):
        tr_loss = torch.cat(tr_losses).mean()
        lo2 = torch.cat(loss2s).mean()
        lolis.append(lo2)
        _in = torch.from_numpy(vin_dense).float().cuda()
        vad_scores, mu, logvar = model.forward(_in)
        vad_loss = models.ae.loss_function(vad_scores, _in, mu, logvar)
        vad_loss.mean().detach().cpu().numpy()
        model.eval()
        model.training=False                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
        wrapper1 = models.ae.implicitWrapper(model, train_data, naive_sparse2tensor, vae=True)
        wrapper2 = models.ae.implicitWrapper(model, vin, naive_sparse2tensor, vae=True)

        
        ret = (tr_loss.detach().cpu().numpy(), vad_loss.mean().detach().cpu().numpy())
        
        for topk in [5, 10]:
            tr_tt[topk].append(ranking_metrics_at_k(wrapper1, empty, train_data, K=topk, num_threads=4)['map'])
            val_tt[topk].append((ranking_metrics_at_k(wrapper2, vin, vo, K=topk, num_threads=4)['map']))
        print(["%0.4f" % x for x in ret], tr_tt[5][-1], tr_tt[10][-1],val_tt[5][-1], val_tt[10][-1])
        qed.append(ret)
add = qed

['294.7231', '225.8089'] 0.21826278174821165 0.17152883630953158 0.05616430555555556 0.05015200877110111
['274.4365', '219.9437'] 0.2750879604178133 0.21030542590868131 0.06868902777777773 0.06192627480158732
['280.2097', '216.5288'] 0.35857614073666827 0.2742169076872701 0.08582458333333337 0.07594075207860913
['275.0313', '214.3504'] 0.400465457210921 0.3105787457333225 0.09942861111111095 0.08769221812799184
['267.6081', '212.9479'] 0.43259849734286077 0.33781472371745475 0.10510222222222197 0.09244149911816571
['263.7416', '212.0777'] 0.4634359538207799 0.3622940053920485 0.1130180555555553 0.09898759511211869
['266.4737', '211.8793'] 0.4883085944658228 0.3832880855266988 0.11566236111111074 0.10196724600025156
['272.7971', '211.2510'] 0.509337548103352 0.401759022361108 0.11879930555555519 0.10581691649344904
['262.5125', '211.0949'] 0.5284835990470935 0.4176413090331327 0.12163013888888845 0.10819690641534364
['274.7474', '210.6121'] 0.5448414879970654 0.4326874071712478 0.122983

['259.1045', '214.6977'] 0.9268123511086737 0.8329643437506576 0.13897249999999972 0.12617133739606934
['265.1940', '214.8335'] 0.9272796408283005 0.8339701823529737 0.13918958333333298 0.12575000543272832
['263.5654', '214.5125'] 0.9297067986072995 0.8383749975414234 0.13796916666666656 0.1252059786785081
['258.7459', '214.9248'] 0.9311599780099016 0.8402338847446248 0.139772083333333 0.12628875015747018
['265.3243', '214.5748'] 0.931131574124983 0.8416007946965607 0.13900611111111094 0.12567849434681252
['257.1082', '214.8861'] 0.934390690855788 0.8463302887354356 0.13806486111111108 0.12572311799256713
['264.7551', '214.9235'] 0.9347544438336146 0.846072154675306 0.13944319444444422 0.1265741059618291
['264.0295', '215.1135'] 0.9377350192413472 0.8498813985615058 0.1396763888888886 0.12661996779730392
['264.2753', '214.9118'] 0.9366208539490622 0.84978738020823 0.1380765277777774 0.12572762117346908
['258.3012', '214.8421'] 0.9376378962800136 0.8499655549741576 0.1372198611111109 0.

In [16]:
lolis

[tensor(-0.4817, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-0.7677, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.0045, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.1744, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.2924, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.3773, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.4989, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.5715, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.5790, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.6430, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.7150, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.7087, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.7785, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.7870, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.8957, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.8911, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.9152, device='cuda:0', grad_f